In [1]:
import sys
from typing import Literal

import numpy as np
import pandas as pd
import plotly.figure_factory as ff
import plotly.graph_objects as go
import tensorflow as tf
from tensorflow.keras.layers import GRU, Dense, Embedding, SimpleRNN, StringLookup
from tensorflow.keras.models import Model

%load_ext autoreload
%autoreload 2

sys.path.append("../")
from equation_discover import *

2023-12-23 19:20:58.195166: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-23 19:20:58.197574: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-23 19:20:58.228975: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-23 19:20:58.229000: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-23 19:20:58.229829: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
n_samples = 32
sampler = EquationSampler(BASE_TOKENS, 16, 2)
(
    sequences,
    entropies,
    log_probs,
    counters,
    lengths,
    sequence_mask,
) = sampler.sample_sequence(n_samples)

In [3]:
X = pd.DataFrame(np.linspace(-2 * np.pi, 2 * np.pi), columns=["var_x"])
y = np.sin((X * 2 + 1).squeeze())

ensemble = ExpressionEnsemble(sequences, lengths)

In [ ]:
expression = ensemble.expressions[21]

In [ ]:
MSE(y, expression.eval(X))

In [ ]:
expression.eval(X)

In [4]:
expression.fit(X, y)

NameError: name 'expression' is not defined

In [6]:
ensemble.fit(X, y)

23-Dec-23 19:22:36 - Fitting 1/32: exp(const) * exp(var_x) - cos(var_x + var_x * var_x), object=ExpressionEnsemble
23-Dec-23 19:22:37 - Fitting 2/32: exp(var_x) * var_x / exp(log(sin(var_x) - var_x)), object=ExpressionEnsemble
23-Dec-23 19:22:37 - Fitting 3/32: var_x - log(var_x + const) * var_x - var_x + var_x, object=ExpressionEnsemble
23-Dec-23 19:22:39 - Fitting 4/32: exp(exp(exp(var_x * var_x + exp(var_x)) + var_x)), object=ExpressionEnsemble
23-Dec-23 19:22:39 - Fitting 5/32: cos(sin(exp(cos(cos(exp(exp(cos(var_x)))))) * var_x)), object=ExpressionEnsemble
23-Dec-23 19:22:39 - Fitting 6/32: exp(sin(sin(exp(cos(exp(var_x * var_x - var_x)))))), object=ExpressionEnsemble
23-Dec-23 19:22:39 - Fitting 7/32: const * exp(cos(exp(cos(const * const))) / var_x), object=ExpressionEnsemble
23-Dec-23 19:22:47 - Fitting 8/32: cos(log(var_x - const)) / var_x - var_x / var_x, object=ExpressionEnsemble
23-Dec-23 19:22:49 - Fitting 9/32: cos(cos(log(var_x / var_x - var_x - var_x) / const)), object=

In [ ]:
ensemble.expressions[0].eval(X)

In [ ]:
Node.from_sequence(sequence[:length], BASE_TOKENS)